In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os
import sys
# Needed for the model class to work inside a Notebook.
os.environ["COBAYA_NOMPI"] = "True"

from cobaya.yaml          import yaml_load_file
from cobaya.samplers.mcmc import plot_progress
from cobaya.model         import get_model
from cobaya.samplers.mcmc import plot_progress
#
from getdist.mcsamples    import MCSamplesFromCobaya
from getdist.mcsamples    import loadMCSamples
import getdist.plots      as     gdplt
#
import os

In [3]:
# Add everything to our paths.
repo = "/global/cscratch1/sd/sfschen/CobayaBAO_DESI/"
for codedir in ["lss_likelihood"]:
    sys.path.append(repo+codedir)

In [4]:
nm = 46
samp = 'z%2d'%(nm)


info = yaml_load_file("recsym_z%2d_pkells_fitdamp.yaml"%(nm))
print(info['params'].keys())
print(info['output'])

dict_keys(['apar', 'aperp', 'Spar', 'Sperp', 'B1_z46', 'F_z46', 'M0_z46', 'M1_z46', 'M2_z46', 'M3_z46', 'M4_z46', 'Q0_z46', 'Q1_z46', 'Q2_z46', 'Q3_z46', 'Q4_z46'])
chains/recsym_z46_pkells_fitdamp


In [5]:
params = {'apar': 0.99959947, 'aperp': 1.0037344000000001, 'Spar': 31.864959000000002, 'Sperp': 12.157199, 'B1_z46': 0.86882982, 'F_z46': 0.7711011999999999, 'M0_z46': -78.399045, 'M1_z46': 1922.5636, 'M2_z46': -1611.251, 'M3_z46': 618.71879, 'M4_z46': -73.817637, 'Q0_z46': 140.92005, 'Q1_z46': -114.50925, 'Q2_z46': 182.09900000000002, 'Q3_z46': -993.71005, 'Q4_z46': 209.7695}

In [6]:
model = get_model(info)
print(list(model.parameterization.sampled_params()))

[model] *WARNING* Ignored blocks/options: ['sampler', 'output', 'stop_on_error']
['apar', 'aperp', 'Spar', 'Sperp', 'B1_z46', 'F_z46', 'M0_z46', 'M1_z46', 'M2_z46', 'M3_z46', 'M4_z46', 'Q0_z46', 'Q1_z46', 'Q2_z46', 'Q3_z46', 'Q4_z46']


In [12]:
model.logposterior(params)
lik_str = 'recon_likelihood.ReconLikelihood'
lik = model.likelihood[lik_str]

tt = lik.bao_predict(samp)
tt = lik.bao_observe(tt)

In [13]:
# Compute templates:
pars_list = params.keys()

In [14]:
err = 0.01
dPs = {}

for param_str in pars_list:
    param = params[param_str]
    
    pars_plus = params.copy()
    pars_plus[param_str] = param + err
    
    pars_minus = params.copy()
    pars_minus[param_str] = param - err

    model.logposterior(pars_plus)
    obs_plus = 1.0 * model.likelihood[lik_str].thy_obs
        
    model.logposterior(pars_minus)
    obs_minus = 1.0 * model.likelihood[lik_str].thy_obs
        
    dPs[param_str] = (obs_plus - obs_minus)/(2*err)

In [15]:
# Dot into covariance to make Fisher matrix
Fij = np.zeros( (len(params.keys()),)*2 )
    
for ii, param_ii in enumerate(pars_list):
    for jj, param_jj in enumerate(pars_list):
            
        Fij[ii,jj] = np.dot(dPs[param_ii],\
                                np.dot(model.likelihood[lik_str].cinv,\
                                       dPs[param_jj]))

In [16]:
np.diag(np.sqrt(np.linalg.inv(Fij)))

<ipython-input-16-7dce64f41c76>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij)))


array([1.91424036e-02, 1.16540282e-02, 2.61520797e+01, 1.67745228e+01,
       1.40673876e-01, 2.93860891e-01, 6.13777669e+01, 1.41482663e+03,
       7.44062743e+02, 3.84815628e+02, 6.89443579e+01, 1.67701760e+02,
       3.24276687e+03, 1.66164654e+03, 7.85334732e+02, 1.37189258e+02])

In [28]:
# Add Priors
iis =  (np.arange(len(pars_list) ) ) > 5 # no apar, aperp, B1, F
prior_inv_sigmas = iis * 1./5000**2

Fij_with_prior = Fij + np.diag(prior_inv_sigmas)

In [29]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))

<ipython-input-29-857f7eb08c5c>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))


array([1.90425466e-02, 1.16372260e-02, 2.54293898e+01, 1.63792506e+01,
       1.27473414e-01, 2.49575024e-01, 5.92582644e+01, 1.35552916e+03,
       7.24596417e+02, 3.80528282e+02, 6.82742743e+01, 1.49442008e+02,
       2.67661976e+03, 1.46726272e+03, 7.49883039e+02, 1.32216416e+02])

In [31]:
corr_mat = np.linalg.inv(Fij_with_prior)/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[None,:]

<ipython-input-31-9a6d4c649bc8>:1: RuntimeWarning: invalid value encountered in sqrt
  corr_mat = np.linalg.inv(Fij_with_prior)/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[None,:]


In [32]:
corr_mat[:6,:6]

array([[ 1.        , -0.3164449 ,  0.15302708, -0.08612452,  0.02423613,
        -0.09930479],
       [-0.3164449 ,  1.        , -0.12314528,  0.12308922, -0.07504992,
         0.05251347],
       [ 0.15302708, -0.12314528,  1.        , -0.36666453, -0.18862297,
         0.39491447],
       [-0.08612452,  0.12308922, -0.36666453,  1.        ,  0.49042223,
        -0.34491771],
       [ 0.02423613, -0.07504992, -0.18862297,  0.49042223,  1.        ,
        -0.70704524],
       [-0.09930479,  0.05251347,  0.39491447, -0.34491771, -0.70704524,
         1.        ]])

In [64]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))

<ipython-input-64-6b31f6b9f5ce>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))


array([0.01297143, 0.00856761, 0.01795289, 0.03575121])

In [65]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:2,:2])))

<ipython-input-65-2c22f3d3ae50>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:2,:2])))


array([0.00429801, 0.0028842 ])

In [66]:
corr_mat = np.linalg.inv(Fij_with_prior[:4,:4])/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[None,:]

<ipython-input-66-800a0b0b04ac>:1: RuntimeWarning: invalid value encountered in sqrt
  corr_mat = np.linalg.inv(Fij_with_prior[:4,:4])/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[None,:]


In [67]:
corr_mat[:4,:4]

array([[ 1.        , -0.36600074,  0.48918734, -0.90307345],
       [-0.36600074,  1.        , -0.93029699,  0.59470038],
       [ 0.48918734, -0.93029699,  1.        , -0.74390488],
       [-0.90307345,  0.59470038, -0.74390488,  1.        ]])